In [32]:
import sys
sys.path.insert(0, '..')
print(sys.path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
print(sys.path)
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode


['..', '..', '..', '..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/ubuntu/.ipython', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/tmp/tmpczg6hdqe', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model']
['..', '..', '..', '..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/l

In [33]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference/test.csv")


In [34]:
clean_df.head()

,Unnamed: 0,test_caption,test_imgs
0,0,feiyangg/LP Paragraph Style Electric Guitar Ti...,O1CN01r2RqZk1lJ4Sk2ZYaN_!!2200643934797.jpg
1,1,Summer Lace White Shorts Pants Female Thin Ant...,O1CN01H0o2pi2HAtGC9rVWa_!!2206736519111.jpg
2,2,High-Quality Export 8MM with Adhesive Cork Cor...,O1CN01qeHW0Z1D3mRPfmNvM_!!161-0-lubanu.jpg
3,3,LED Ceiling Living Room Bedroom Corridor Corri...,O1CN01phQoOe25ljjN1lpLO_!!3082327567.jpg
4,4,Disposable Cotton Cotton Female Thin Water Fac...,O1CN01Dzpgpr2FxJsUzqAdE_!!2963188946.jpg


In [35]:
clean_df.tail()

,Unnamed: 0,test_caption,test_imgs
49995,49995,this data is null,TB1gz7TgipE_u4jSZKbq6zCUVXa
49996,49996,this data is null,O1CN018DjGnX1QeV9jOECs2_!!3432982001.jpg
49997,49997,this data is null,O1CN019QiqKF1MnQTM0gdna_!!2211026631479.jpg
49998,49998,this data is null,TB1yui5VoT1gK0jSZFrq6ANCXXa
49999,49999,this data is null,O1CN01cxGSZT26bwr7f1io2_!!2707697681.jpg


In [36]:
# col_to_test =  [c for c in clean_df.columns.values if "caption" in c] + [c for c in clean_df.columns.values if "step" in c]
col_to_test =  "test_caption"

In [37]:
image_size = 384
preprocess = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 


In [38]:
def gen_embed(caption_col,clip_model):
    test_loader,df = prepare_data("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference/test.csv",
    CONFIG.test_image_data_dir,"test_imgs"
    ,"test_imgs",caption_col,160,preprocess,CONFIG.global_random_state,test=True,tokenize=False)
    trainer = Trainer(gpus=1)
    pred = trainer.predict(clip_model,test_loader)
    full_pred = tuple(map(torch.concat, zip(*pred)))
    image_embed,text_embed = full_pred

    return image_embed,text_embed

In [39]:
def save_embed(image_embed,text_embed,model_size:str,prefix:str):
    img_embed_path = "./image_embeds"
    txt_embed_path = "./text_embeds"
    img_embed_filename = f"img_emded-{model_size}-{prefix}.pkl"
    txt_embed_filename = f"txt_emded-{model_size}-{prefix}.pkl"
    img_embed_full_path = f"{img_embed_path}/{img_embed_filename}"
    txt_embed_full_path = f"{txt_embed_path}/{txt_embed_filename}"

    image_embed_info_dict = {"model_name":prefix,"model_size":model_size,"pair_name":txt_embed_filename,"embed":image_embed}
    text_embed_info_dict = {"model_name":prefix,"model_size":model_size,"pair_name":img_embed_filename,"embed":text_embed}

    with open(img_embed_full_path,"wb") as file:
        print(f"saving img embed from {prefix} to {img_embed_full_path}")
        pickle.dump(image_embed_info_dict,file)
    with open(txt_embed_full_path,"wb") as file:
        print(f"saving txt embed from {prefix} to {txt_embed_full_path}")
        pickle.dump(text_embed_info_dict,file)
    

In [40]:
models = [
    {"path":"/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model/BLIP/output/save_checkpoint_7_thu.pth",
    "model_size":"base","model_name":"base_all_e_7"},
    {"path":"/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model/BLIP/output/Retrieval_aliproduct2_filtered/save_checkpoint_4.pth",
    "model_size":"base","model_name":"base_1st_e_5"},
    {"path":"/home/ubuntu/Desktop/base_2nd/base_2nd_e_5.pth","model_size":"base","model_name":"base_2nd_e_5"},
    {"path":"/home/ubuntu/Desktop/base_3_itm/base_3rd_e_5.pth",
    "model_size":"base","model_name":"base_3rd_e_5"},
     {"path":"/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_8.pth","model_size":"large","model_name":"large_all_e_9"},
    {"path":"/home/ubuntu/Desktop/large_v4/save_checkpoint_4.pth","model_size":"large","model_name":"large_v1_e_5"},
    ]

for model in models:
    print(f"extracting embeddings from "+model["model_name"])
    clip_model =ALIPRODUCT_BLIP(model["path"],image_size,vit=model["model_size"])
    image_embed,text_embed = gen_embed(col_to_test,clip_model)
    text_embed_no_null = text_embed[0:20000]
    save_embed(image_embed,text_embed_no_null,model["model_size"],model["model_name"])
    


extracting embeddings from base_all_e_7


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


load checkpoint from /home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model/BLIP/output/save_checkpoint_7_thu.pth


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 313/313 [18:39<00:00,  3.58s/it]
saving img embed from base_all_e_7 to ./image_embeds/img_emded-base-base_all_e_7.pkl
saving txt embed from base_all_e_7 to ./text_embeds/txt_emded-base-base_all_e_7.pkl
extracting embeddings from base_1st_e_5


...

Predicting DataLoader 0: 100%|██████████| 313/313 [52:20<00:00, 10.04s/it]
saving img embed from large_all_e_9 to ./image_embeds/img_emded-large-large_all_e_9.pkl
saving txt embed from large_all_e_9 to ./text_embeds/txt_emded-large-large_all_e_9.pkl
extracting embeddings from large_v1_e_5


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


load checkpoint from /home/ubuntu/Desktop/large_v4/save_checkpoint_4.pth


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 313/313 [52:20<00:00, 10.03s/it]
saving img embed from large_v1_e_5 to ./image_embeds/img_emded-large-large_v1_e_5.pkl
saving txt embed from large_v1_e_5 to ./text_embeds/txt_emded-large-large_v1_e_5.pkl
